# Metadata extraction/filtering

In certain scenarios, RAG benefits from filtering database queries using metadata. This is one of those scenarios. The knowledge base is annual reports from Microsoft, Alphabet, and Meta for the years 2020-2023. We'll chunk the reports, insert the chunks into a ChromaDB database, and tag each chunk with metadata indicating the company name and the year of the report. When asked a question such as "What was Microsoft's revenue in 2022," the app will query the database for relevant chunks where company equals "microsoft" and year equals "2022." This way, the app can provide accurate answers without mixing up annual reports from different companies and different years. We'll use an LLM to determine what metadata values, if any, to use for filtering.

Start by creating a ChromaDB database in the "chroma" subdirectory and creating a collection named "Annual_Reports:"

In [1]:
import chromadb

client = chromadb.PersistentClient('chroma')
collection = client.create_collection(name='Annual_Reports')

Now use LLamaIndex's `DocxReader` and `PdfReader` classes to extract text from the annual reports in the "Data" subdirectory and LlamaIndex's `SentenceSplitter` class to chunk the text. Then insert the chunks into the database along with metadata identifying the company and the year:

In [2]:
import os, re
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.file.docs import DocxReader, PDFReader

path = 'Data'
pattern = re.compile(r"^(?P<company>\w+)_(?P<year>\d{4})\.(?P<extension>\w+)$")

for filename in os.listdir(path):
    match = pattern.match(filename)

    if match:
        company = match.group("company").lower()
        year = match.group("year").lower()
        extension = match.group("extension").lower()
        full_path = os.path.join(path, filename)

        if extension == 'pdf':
            reader = PDFReader()
        elif extension == 'docx':
            reader = DocxReader()
        else:
            continue

        print(f'Processing {filename}...')
        document = reader.load_data(full_path)
        splitter = SentenceSplitter(chunk_size=512, chunk_overlap=40)
        nodes = splitter.get_nodes_from_documents(document)

        for i, node in enumerate(nodes):
            if len(node.text) > 128:
                collection.add(
                    documents=[node.text],
                    metadatas=[{ 'company': company, 'year': year }],
                    ids=[f'{company}-{year}-{i:05}']
                )

Processing Alphabet_2020.pdf...
Processing Alphabet_2021.pdf...
Processing Alphabet_2022.pdf...
Processing Alphabet_2023.pdf...
Processing Meta_2020.pdf...
Processing Meta_2021.pdf...
Processing Meta_2022.pdf...
Processing Meta_2023.pdf...
Processing Microsoft_2020.docx...
Processing Microsoft_2021.docx...
Processing Microsoft_2022.docx...
Processing Microsoft_2023.docx...


Define a function to extract metadata values from a question and return a JSON structure that can be used in a `where` clause in a ChromaDB query:

In [3]:
from openai import OpenAI

def extract_metadata(question):
    content = f'''
        The question below will be used to query a vector database. Each item in the database
        has metadata values named "company" and "year" designating the company that the item
        pertains to and the year it was recorded. From the question, generate JSON that
        indicates which, if any, metadata values should be included in the query and what
        values to assign to them. If the company name is "Google," use "Alphabet" instead.
        If the company name is "Facebook," use "Meta" instead.
        
        If a company name is detected but a year is not, use this format:
    
        {{
            "company": "value"
        }}

        If a year is detected but a company name is not, use this format:
    
        {{
            "year": "value"
        }}

        If both a company name and a year are detected, use this format:

        {{
            "$and": [
                "company": "value",
                "year": "value"
            ]
        }}

        Do not return markdown. Only return JSON. All JSON values must be strings.
        
        Question:
        {question}
        '''

    messages = [{ 'role': 'user', 'content': content }]
    client = OpenAI(api_key='OPENAI_API_KEY')

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        response_format={ 'type': 'json_object' }
    )

    return response.choices[0].message.content

Test the function with a question that contains two metadata values:

In [4]:
print(extract_metadata("What was Microsoft's revenue in 2022?"))

{"$and": [{"company": "Microsoft"}, {"year": "2022"}]}


Test it with a question that contains one metadata value:

In [5]:
print(extract_metadata("Who is Google's CEO?"))

{"company": "Alphabet"}


Test it with a question that contains no metadata values:

In [6]:
print(extract_metadata('What is sustainability?'))

{}


Load `jina-reranker-v1-turbo-en` for reranking query results:

In [7]:
from sentence_transformers import CrossEncoder

model = CrossEncoder('jinaai/jina-reranker-v1-turbo-en', trust_remote_code=True)

C:\Users\jeffp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Define a function for asking questions about the annual reports:

In [8]:
import json

def answer_question(question):
    client = chromadb.PersistentClient('chroma')
    collection = client.get_collection(name='Annual_Reports')    

    # Extract metadata from question
    metadata = extract_metadata(question).lower()

    # Query the database
    results = collection.query(
        query_texts=[question],
        where=json.loads(metadata),
        n_results=40
    )
    
    # Exit early if metadata filtering caused the query to return no results
    documents = results['documents'][0]

    if len(documents) == 0:
        print("I don't know.")
        return
        
    # Rerank the results
    ranked_documents = model.rank(question, documents, return_documents=True, top_k=20)
    context = '\n\n'.join(x['text'] for x in ranked_documents)

    # Submit the question and the chunks to an LLM and stream the response
    client = OpenAI(api_key='OPENAI_API_KEY')

    content = f'''
        Answer the following question using the provided context, and if the
        answer is not contained within the context, say "I don't know." Explain
        your answer if possible. Do not mention the provided context in your
        output. Do not use markdown formatting. Do not use backslash characters.
        
        Question:
        {question}

        Context:
        {context}
        '''

    messages = [{ 'role': 'user', 'content': content }]

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        stream=True
    )

    for chunk in response:
        content = chunk.choices[0].delta.content
        if content is not None:
            print(content, end='')  

Ask a question regarding Microsoft's 2022 revenue:

In [9]:
answer_question("What was Microsoft's revenue in 2022?")

Microsoft's revenue in 2022 was $198.27 billion.

Do the same for Google in 2023:

In [10]:
answer_question("What was Google's revenue in 2023?")

Google's revenue in 2023 was $307.4 billion.

Ask a question that contains just one metadata value:

In [11]:
answer_question("How important is diversity at Facebook?")

Diversity is of significant importance at Facebook. The company explicitly states that diversity and inclusion are core to its mission. It aims to create a diverse and inclusive workplace to harness cognitive diversity for better product development and decision-making. Facebook has set ambitious diversity goals, including having 50% of its workforce comprised of underrepresented groups by 2024 and increasing the representation of people of color in leadership roles. Various initiatives are in place to support these goals, including recruiting strategies, training programs, and events aimed at building community and professional development for underrepresented groups.

Ask a question involving a company not represented in the database:

In [12]:
answer_question("What was Twitter's revenue in 2022?")

I don't know.
